In [16]:
#Google Sheet Link: https://docs.google.com/spreadsheets/d/1RiU9c3YihCsAqh18C25WMfTxkOiAf-nXhxAWtEaKLA0/edit#gid=0
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 100) #Displays a max of 500 rows for testing

sheet_id = '1RiU9c3YihCsAqh18C25WMfTxkOiAf-nXhxAWtEaKLA0'
xls = pd.ExcelFile(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx")

arrivals = pd.read_excel(xls, 'Arrivals', header = 0) #Reads in the Google Sheets Data
arrivals = arrivals.drop(['blank', 'Communication Type/Reason for Speaking.1'], axis=1) #Drops unnecessary Columns/Headers
arrivals = arrivals.dropna(thresh=14) #Drops all rows and columns with more than 14 N/A values
departures = pd.read_excel(xls, 'Departures', header = 0)
departures = departures.drop(['Airport', 'Communication Type/Reason for Speaking.1'], axis=1)

#You'll need this 'pip install -U scikit-learn scipy matplotlib' to install it, or 'pip3 install -U scikit-learn scipy matplotlib' for Python3
from sklearn.ensemble import RandomForestClassifier 
rf = RandomForestClassifier()
#rf.fit()
#RandomForestClassifier()